In [3]:
#import config.py that lists out twitter developer api keys to secure a connection between twitter and python app
from config import *
import tweepy as tw
import pandas as pd
import csv

In [4]:
#file open
file = open('/Users/aj/Downloads/sephora_website_dataset.csv')
type(file)
#/Users/aj/Downloads/sephora_website_dataset.csv

_io.TextIOWrapper

In [1]:
#unique brands that are in face category 
brands = open('/Users/aj/Downloads/brandindex.csv')

In [4]:
#put it into dataframe
brands = pd.read_csv(brands)
brands.head()

,brand_index,brand_name
0,1,AERIN
1,2,Aether Beauty
2,3,Algenist
3,4,Alpha-H
4,5,alpyn beauty


In [5]:
# reset brand_index as table index
brands = brands.set_index('brand_index')

In [7]:
# assign unique brand names into totalbrands as a list 
totalbrands = brands['brand_name'].unique().tolist()

In [8]:
# check length of totalbrands
len(totalbrands)

186

In [90]:
#Authentification process for twitter (tweepy api)
auth = tw.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
#Iterate over a list of brands and save the tweeter data into dataframe
numberOfTweets = 1500
tweet_dataset = pd.DataFrame(columns=['Brand','Tweet Id', 'Tweet Date', 'Follower Count','Retweets', 'Location', 'Tweet Text'])
for x in totalbrands: #goes through each brand as hashtag keyword
    for tweet in tw.Cursor(api.search_tweets, tweet_mode='extended', q=x, lang="en").items(numberOfTweets):
        appending_dataframe = pd.DataFrame([[x, tweet.id, tweet.created_at, tweet.user.followers_count, tweet.retweet_count, tweet.user.location, tweet.full_text.encode('utf-8')]], columns=['Brand','Tweet Id', 'Tweet Date', 'Follower Count', 'Retweets','Location','Tweet Text'])
        tweet_dataset = tweet_dataset.append(appending_dataframe)

In [ ]:
#Create a dictionary for brands so that brand names can be changed into brand index to link with brands table 
mapping_dict = brands.to_dict()

In [162]:
tweet_dataset

,Brand,Tweet Id,Tweet Date,Follower Count,Retweets,Location,Tweet Text
0,AERIN,1460799974095015939,2021-11-17 02:40:28+00:00,81,0,San Diego,"b""@RTCath95 he's awffully strange aint he?"""
0,AERIN,1460799337655590919,2021-11-17 02:37:57+00:00,595,0,テキサス,b'@Just_Like_Aerin Lyse is a solid not entirel...
0,AERIN,1460798057801392131,2021-11-17 02:32:51+00:00,81,0,San Diego,"b""@SNovatore true let's homosexually terrorize..."
0,AERIN,1460797365879590916,2021-11-17 02:30:06+00:00,1042,0,,b'@spaceshinji ngl if I were to go with someth...
0,AERIN,1460796093441232900,2021-11-17 02:25:03+00:00,595,0,テキサス,b'@Just_Like_Aerin Alphinaud is your new middl...
...,...,...,...,...,...,...,...
0,Yves Saint Laurent,1460180104194129923,2021-11-15 09:37:20+00:00,1491,0,she/her — 17,"b'besties, how do i pronounce yves saint laure..."
0,Yves Saint Laurent,1460180068362190849,2021-11-15 09:37:11+00:00,4809,0,"New York, NY",b'\xf0\x9f\x91\x89 $72.72 \xf0\x9f\x91\x88\nYv...
0,Yves Saint Laurent,1460171537810558983,2021-11-15 09:03:17+00:00,879,16,,b'RT @schiaplicious: Catherine Deneuve as Seve...
0,Yves Saint Laurent,1460166008887558148,2021-11-15 08:41:19+00:00,6898,0,she/her | 21,"b""It's a story/study on Yves Saint Laurent and..."


In [20]:
#Switch the key and value to fit the needs
my_dict = {y:x for x,y in mapping_dict.items()}

In [21]:
#Apply dictionary and change brand name to matching brand index 
tweet_dataset2 = tweet_dataset.replace({"Brand": my_dict})

In [181]:
#
tweet_dataset2 = tweet_dataset2.rename(columns={"Brand": "brand_id", 'Tweet Id': 'tweet_id', 'Tweet Date': 'tweet_date', 'Follower Count': 'follower_count','Retweets': 'retweets', 'Location':'location', 'Tweet Text':'tweet_text'
                                            })

In [191]:
#Drop duplicate data
tweet_dataset2.drop_duplicates(keep=False,inplace=True)
#Reorder tweet_dataset dataset columns 
tweet_dataset2 = pd.DataFrame(tweet_dataset2,columns=['tweet_id',"brand_id",'tweet_date','follower_count','retweets','location','tweet_text'])
tweet_dataset2

,tweet_id,brand_id,tweet_date,follower_count,retweets,location,tweet_text
0,1460799974095015939,1,2021-11-17 02:40:28+00:00,81,0,San Diego,"b""@RTCath95 he's awffully strange aint he?"""
0,1460799337655590919,1,2021-11-17 02:37:57+00:00,595,0,テキサス,b'@Just_Like_Aerin Lyse is a solid not entirel...
0,1460798057801392131,1,2021-11-17 02:32:51+00:00,81,0,San Diego,"b""@SNovatore true let's homosexually terrorize..."
0,1460797365879590916,1,2021-11-17 02:30:06+00:00,1042,0,,b'@spaceshinji ngl if I were to go with someth...
0,1460796093441232900,1,2021-11-17 02:25:03+00:00,595,0,テキサス,b'@Just_Like_Aerin Alphinaud is your new middl...
...,...,...,...,...,...,...,...
0,1460180104194129923,186,2021-11-15 09:37:20+00:00,1491,0,she/her — 17,"b'besties, how do i pronounce yves saint laure..."
0,1460180068362190849,186,2021-11-15 09:37:11+00:00,4809,0,"New York, NY",b'\xf0\x9f\x91\x89 $72.72 \xf0\x9f\x91\x88\nYv...
0,1460171537810558983,186,2021-11-15 09:03:17+00:00,879,16,,b'RT @schiaplicious: Catherine Deneuve as Seve...
0,1460166008887558148,186,2021-11-15 08:41:19+00:00,6898,0,she/her | 21,"b""It's a story/study on Yves Saint Laurent and..."
